In [2]:
import torch
import pandas as pd
import numpy as np

### Load data file containing all info (indexes, ddb, Lipschitz, etc)

In [3]:
data = torch.load('../KD_data/CIFAR_fas_cumm.pth')
df = pd.DataFrame(data)
# df = pd.DataFrame.from_dict(data, orient='index')
# df = df.transpose()

for k, v in df.items():
    if type(v) == list:
        print(f'{k} \t {len(v)}')
    else:
        print(f'{k}  \t {v.shape}')
df.head()

idx  	 (50000,)
ddbs  	 (50000,)
gt  	 (50000,)
steps_to_atk  	 (50000,)
predictions  	 (50000,)
Flipping_Freq  	 (50000,)
Model_preds  	 (50000,)
lip  	 (50000,)


,idx,ddbs,gt,steps_to_atk,predictions,Flipping_Freq,Model_preds,lip
0,0,1.552896,6,1,5,23,6,14.552392
1,1,1.621443,9,2,1,17,9,12.178064
2,2,1.473749,9,1,7,17,9,12.254439
3,3,1.443467,4,1,7,14,4,19.642838
4,4,1.576641,1,4,9,14,1,5.591359


### Choose top 6250 based on Lipschitz value (higher is less robust)

In [16]:
top_n = 6250
df_s,idx_f = {},{}
idx_kd = [False]*len(df)

df_s['lip'] = df.sort_values(by=['lip'], ascending=False)[:top_n]
idx_f['lip'] = np.array(df_s['lip']['idx'].to_list())
for val in idx_f['lip']:
    idx_kd[val] = True

np.save(f'../KD_data/lip_selected_{top_n}.npy',idx_kd )
df_s['lip']

,idx,ddbs,gt,steps_to_atk,predictions,Flipping_Freq,Model_preds,lip
29415,29415,1.406058,2,1,7,3,2,57.317440
41902,41902,1.323325,8,1,9,1,8,57.076374
28016,28016,1.313783,8,1,4,1,8,50.525368
37176,37176,1.389575,3,1,5,20,3,50.524387
23483,23483,1.350177,3,1,6,11,3,47.676983
...,...,...,...,...,...,...,...,...
41715,41715,1.385836,3,1,6,17,3,20.178356
25532,25532,1.347520,0,1,2,3,0,20.177448
45135,45135,1.515434,6,2,3,18,6,20.177410
9301,9301,1.474313,5,1,3,9,5,20.176834


### Choose top 6250 based on ddbs value (lower is less robust)

In [17]:
top_n = 6250
df_s,idx_f = {},{}
idx_kd = [False]*len(df)

df_s['ddbs'] = df.sort_values(by=['ddbs'], ascending=False)[:top_n]
idx_f['ddbs'] = np.array(df_s['ddbs']['idx'].to_list())

for val in idx_f['ddbs']:
    idx_kd[val] = True

np.save(f'../KD_data/ddbs_selected_{top_n}.npy',idx_kd )
df_s['ddbs']

,idx,ddbs,gt,steps_to_atk,predictions,Flipping_Freq,Model_preds,lip
29920,29920,1.707418,2,0,6,29,2,6.947077
4289,4289,1.702838,9,3,1,14,9,7.273105
39133,39133,1.698200,6,2,1,21,6,7.189980
31866,31866,1.697402,4,0,6,22,4,5.024558
28821,28821,1.697245,1,5,9,11,1,7.320117
...,...,...,...,...,...,...,...,...
20915,20915,1.608845,9,1,1,14,9,10.241366
16153,16153,1.608830,6,1,4,7,6,21.032518
13457,13457,1.608825,0,2,2,18,0,10.467146
43626,43626,1.608821,3,1,6,20,3,14.408295


### Classwise

In [14]:
num_classes = 10
top_n_cls = 650
df_filter_sort = []
df_copy = df.copy()

for i in range(num_classes):
    df_filter = df_copy[(df_copy['gt'] == i)]
    df_filter_sort[i] = df_filter.sort_values(by=['lip'], ascending=True)[:top_n_cls]
    df_id = df_filter_sort[i]['idx'].to_list()
    df_copy = df_copy.drop(labels=df_id, axis=0)
